Reference :
Dataset : https://github.com/rgbnihal2/COVID-19-X-ray-Dataset



In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model

import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
#device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


Now we'll rearrange the images. Devide them into three folders train, validation, test.

In [ ]:
os.chdir('E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master')
if os.path.isdir('train/covid') is False:
    os.makedirs('train/covid')
    os.makedirs('train/normal')
    os.makedirs('valid/covid')
    os.makedirs('valid/normal')
    os.makedirs('test/covid')
    os.makedirs('test/normal')
    

We'll copy images to related folders. First we'll maintain a ration of, <br>
80% train, 10% val, 10% test. <br>

In [ ]:
os.chdir('E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master')

for i in random.sample(glob.glob('E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/COVID_Test*'),219):
    shutil.move(i, 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/train/covid')
for i in random.sample(glob.glob('E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/NORMAL_Test*'),219):
    shutil.move(i, 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/train/normal')
for i in random.sample(glob.glob('E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/COVID_Test*'), 28):
    shutil.move(i, 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/valid/covid')
for i in random.sample(glob.glob('E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/NORMAL_Test*'), 28):
    shutil.move(i, 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/valid/normal')
for i in random.sample(glob.glob('E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/COVID_Test*'), 26):
    shutil.move(i, 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/test/covid')
for i in random.sample(glob.glob('E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/NORMAL_Test*'), 26):
    shutil.move(i, 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/test/normal')

In [5]:
train_path = 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/train'
valid_path = 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/valid'
test_path = 'E:/DeepLearning/Datasets/COVID-19-X-ray-Dataset-master/test'

In [6]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['covid', 'normal'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['covid', 'normal'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['covid', 'normal'], batch_size=10, shuffle=False)

Found 438 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
Found 52 images belonging to 2 classes.


In [ ]:
imgs, labels = next(train_batches)


In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
plotImages(imgs)
print(labels)

In [ ]:
model = Sequential([
    Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(x=train_batches,
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=10,
    verbose=2
)

### Predict


In [ ]:
test_imgs, test_labels = next(test_batches)

In [ ]:
test_batches.classes

In [ ]:
plotImages(test_imgs)
print(test_labels)

In [ ]:
predictions = model.predict(x = test_batches, verbose=0)

In [ ]:
np.round(predictions)

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred= np.argmax(predictions, axis= -1)  )


In [ ]:

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['covid','normal']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
model.save('models/Covid19_Classification_from_X-RAY _Images_normal_model.h5')


### Train with MobileNet


In [6]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['covid', 'normal'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['covid', 'normal'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['covid', 'normal'], batch_size=10, shuffle=False)

Found 438 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
Found 52 images belonging to 2 classes.


In [12]:
IMG_SHAPE = (224, 224, 3) 
base_model = tf.keras.applications.MobileNetV2( #grabbing pretrained neural network of choice
    input_shape=IMG_SHAPE,
    include_top=False, #this will freeze all the weights, because we dont have to retrain and change the weights, instead just add on to the MobileNetV2 CNN, so it clasiffies 5 classes instead of 80
    weights='imagenet'
)

9412608/9406464 [==============================] - 6s 1us/step


In [14]:
base_model.trainable=False #this freezes all the neurons for our base model


In [15]:
model = tf.keras.Sequential([ #neural networks act in a sequence of layers, so we add layers as we want
  base_model,
  tf.keras.layers.Conv2D(32,3, activation = 'relu'), #This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. Bascially, it trying to understand the patterns of the image
  tf.keras.layers.Dropout(0.2), #This layer prevents Neural Networks from Overfitting, i.e being too precise to a point where the NN is only able to recognize images that are present in the dataset
  tf.keras.layers.GlobalAveragePooling2D(), #This layer calculates the average output of each feature map in the previous layer, thus reducing the data significantly and preparing the model for the final layer
  tf.keras.layers.Dense(2, activation='softmax')
])

In [15]:
model = Model(inputs=mobile.input, outputs=output)

In [16]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:
model.fit(x=train_batches,
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=30,
            verbose=2
)

Epoch 1/30
44/44 - 30s - loss: 0.1840 - accuracy: 0.9201 - val_loss: 0.0178 - val_accuracy: 1.0000
Epoch 2/30
44/44 - 14s - loss: 0.0292 - accuracy: 0.9909 - val_loss: 0.0411 - val_accuracy: 0.9821
Epoch 3/30
44/44 - 13s - loss: 0.0104 - accuracy: 0.9977 - val_loss: 0.0067 - val_accuracy: 1.0000
Epoch 4/30
44/44 - 14s - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0125 - val_accuracy: 1.0000
Epoch 5/30
44/44 - 13s - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0077 - val_accuracy: 1.0000
Epoch 6/30
44/44 - 13s - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 7/30
44/44 - 13s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 8/30
44/44 - 13s - loss: 8.5894e-04 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 9/30
44/44 - 13s - loss: 6.5896e-04 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 10/30
44/44 - 13s - loss: 5.9360e-04 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 1.00